In [ ]:
# import csv

# # Open original and new CSV files
# with open('slogon.csv', 'r', encoding='utf-8') as infile, open('new_slogon.csv', 'w', newline='', encoding='utf-8') as outfile:
#     reader = csv.reader(infile)
#     writer = csv.writer(outfile)
    
#     # Write new header
#     writer.writerow(['prompt', 'slogon'])

#     next(reader)  # Skip the header line

#     for row in reader:
#         try:
#             # Example row: ['logo_00000', 'a logo...', 'slogan...']
#             # If your row only has 1 item (i.e., full line), split it manually
#             if len(row) == 1:
#                 parts = row[0].split(',"')
#                 if len(parts) == 3:
#                     prompt = parts[1].strip('"')
#                     slogon = parts[2].strip('"')
#                     writer.writerow([prompt, slogon])
#             elif len(row) >= 3:
#                 # Already split correctly
#                 writer.writerow([row[1], row[2]])
#         except Exception as e:
#             print(f"Skipping malformed row: {row} — Error: {e}")


In [1]:
import pandas as pd

# Load CSV with correct delimiter
df = pd.read_csv('new_slogon.csv', delimiter=',')

# Get and print column names
print(df.columns.tolist())

['prompt', 'slogon']


In [2]:
df.head()

,prompt,slogon
0,"a logo of coffee shop, take-away coffee cardbo...","Fuel Every Moment, One Sip at a Time"
1,"a logo of coffee shop, White round background ...",Brew & Bite: Where Warmth Meets Delight
2,"a logo of coffee shop, image of a filled cup w...","Rise and Shine, Sip by Sip"
3,a logo of cafe restaurant bar pizzeria with a ...,Where Every Slice Marks a New Chapter
4,a logo of cafe restaurant bar with a circle wi...,"Gather to Savor, Return for the Flavor"


In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import pandas as pd

# Load your dataset
df = pd.read_csv("new_slogon.csv")

# Clean up any rows with missing values
df = df.dropna()

# Rename columns to match expected format
df = df.rename(columns={"prompt": "input_text", "slogon": "target_text"})

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Split into train and validation
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
val_dataset = dataset["test"]




In [2]:
# Load tokenizer and model
model_checkpoint = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Tokenize the inputs and targets
max_input_length = 64
max_target_length = 32

def preprocess(batch):
    inputs = tokenizer(batch["input_text"], max_length=max_input_length, truncation=True, padding="max_length")
    targets = tokenizer(batch["target_text"], max_length=max_target_length, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs


tokenized_train = train_dataset.map(preprocess, batched=True)
tokenized_val = val_dataset.map(preprocess, batched=True)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


Map:   0%|          | 0/722 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

In [3]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./slogan-generator",
    eval_strategy="epoch",              # Evaluate after each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,            # Reduce if you face memory issues
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,                      # 🔼 Increase from 4 → 10
    predict_with_generate=True,
    logging_dir="./logs",
    logging_strategy="steps",                 # Better logging visibility
    logging_steps=100,                        # Log every 100 steps
    save_strategy="epoch",
    report_to="none",                         # Disable external logging
    generation_max_length=32,                 # Ensure generation is controlled
    generation_num_beams=5                    # Use beam search during evaluation
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


C:\Users\nisha\AppData\Local\Temp\ipykernel_4396\3316486813.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,16.454000,1.465001
2,1.566500,0.745006
3,0.820000,0.667785
4,0.722700,0.645550
5,0.680100,0.633906
6,0.645200,0.628949
7,0.636200,0.626490
8,0.632100,0.622929
9,0.609400,0.622389
10,0.606400,0.622057


TrainOutput(global_step=1810, training_loss=1.7612010176010553, metrics={'train_runtime': 2501.7657, 'train_samples_per_second': 2.886, 'train_steps_per_second': 0.723, 'total_flos': 617993510584320.0, 'train_loss': 1.7612010176010553, 'epoch': 10.0})

In [4]:
trainer.save_model("./slogan-generator/checkpoint-final")


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("./slogan-generator/checkpoint-final")

# Generate slogan
prompt = "logo of beauty spa with a face silhouette and lotus flower inside a soft pink elli"
input_text = "Generate a slogan for: " + prompt
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Generate output
outputs = model.generate(**inputs, max_length=32, num_beams=5)
slogan = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("📝 Slogan:", slogan)


📝 Slogan: Elegance in Every Face


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("./slogan-generator/checkpoint-final")

# Prompt
prompt = "logo of beauty spa with a face silhouette and lotus flower inside a soft pink elli"
input_text = "Generate a slogan for: " + prompt
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Generate 3 different slogans
outputs = model.generate(
    **inputs,
    max_length=32,
    num_return_sequences=3,
    do_sample=True,         # Enable sampling for diversity
    top_k=50,               # Limit to top 50 tokens
    temperature=0.9         # Add randomness (higher = more random)
)

# Decode and print slogans
for i, output in enumerate(outputs, 1):
    slogan = tokenizer.decode(output, skip_special_tokens=True)
    print(f"📝 Slogan {i}:", slogan)


📝 Slogan 1: Laugh on the Best
📝 Slogan 2: Elegance and Beauty
📝 Slogan 3: Smile with Love


In [ ]:
dawdasjk ck

In [1]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    print("CUDA not available. Running on CPU.")


PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1
GPU name: NVIDIA GeForce RTX 3050 Laptop GPU


In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(0))

CUDA Available: True
Current Device: 0
Device Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [ ]:
from flask import Flask, render_template, request, jsonify
from PIL import Image
import io
import base64
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import os

app = Flask(__name__)

# Initialize models (could be moved to a separate service for better performance)
def init_models():
    # Slogan generator
    slogan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
    slogan_model = AutoModelForSeq2SeqLM.from_pretrained("./slogan_model/checkpoint-final")
    
    # Logo generator
    logo_pipe = StableDiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base",
        torch_dtype=torch.float16
    ).to("cuda")
    
    # Load LoRA weights if available
    adapter_path = "./logo_model/checkpoint-94000"
    weight_file = "pytorch_lora_weights.safetensors"
    if os.path.exists(os.path.join(adapter_path, weight_file)):
        logo_pipe.load_lora_weights(adapter_path, weight_name=weight_file)
    
    logo_pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(logo_pipe.scheduler.config)
    
    return slogan_tokenizer, slogan_model, logo_pipe

slogan_tokenizer, slogan_model, logo_pipe = init_models()

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    prompt = data['prompt']
    
    # Generate slogans
    input_text = "Generate a slogan for: " + prompt
    inputs = slogan_tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    
    slogan_outputs = slogan_model.generate(
        **inputs,
        max_length=32,
        num_return_sequences=3,
        do_sample=True,
        top_k=50,
        temperature=0.9
    )
    
    slogans = [slogan_tokenizer.decode(output, skip_special_tokens=True) 
               for output in slogan_outputs]
    
    # Generate logos
    negative_prompt = "low quality, worst quality, bad composition, extra digit, fewer digits, text, inscription, watermark, label, asymmetric"
    
    logo_images = []
    for _ in range(3):
        result = logo_pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=30,
            guidance_scale=7.5,
            height=512,
            width=512
        )
        img = result.images[0]
        
        # Convert to base64 for web display
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
        logo_images.append(img_str)
    
    return jsonify({
        'slogans': slogans,
        'logos': logo_images
    })

if __name__ == '__main__':
    app.run(debug=True)